In [1]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.{col}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
142,application_1576053140655_0143,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.col
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


In [2]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [3]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


In [4]:
// convert the label from {0, 1} to {1, 2}
//val labelConverter = new FuncTransformer(udf {d: Double => if (d==0) 2 else d }).setInputCol("Class").setOutputCol("Class")
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
//val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_73a67da6c209
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_c27d2d7772d7
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_a681267ba94b
pipeline: org.apache.spark.ml.Pipeline = pipeline_c27d72a76824
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_c27d72a76824
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.88136490328633...|  0.0|
|[0.84029849039390...|  0.0|
|[0.86814081926190...|  0.0|
|[0.86848364774806...|  0.0|
|[0.86425070140685...|  0.0|
|[0.85718742663956...|  0.0|
|[0.83819983804773...|  0.0|
|[0.85603110871019...|  0.0|
|[0.83545216735689...|  0.0|
|[0.85551101187934...|  0.0|
|[0.85324951634142...|  0.0|
|[0.82226325870284...|  0.0|
|[0.84406658980857...|  0.0|
|[0.85177230541965...|  0.0|
|[0.86586281889129

### step 2. split the dataset into training and validation dataset.

In [5]:
val data0 = data.filter($"Class" === "0").cache()
val data1 = data.filter($"Class" === "1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
data1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [6]:
val splitTime0 = data0.stat.approxQuantile("Time", Array(0.7), 0.001).head
val splitTime1 = data1.stat.approxQuantile("Time", Array(0.4), 0.001).head
val trainingData0 = data0.filter(s"Time<$splitTime0").cache()
val validationData0 = data0.filter(s"Time>=$splitTime0").cache()
val trainingData1 = data1.filter(s"Time<$splitTime1").cache()
val validationData1 = data1.filter(s"Time>=$splitTime1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime0: Double = 132882.0
splitTime1: Double = 58642.0
trainingData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
trainingData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [7]:
trainingData0.select("V1", "Class").show(10)
trainingData1.select("V1", "Class").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+
|                V1|Class|
+------------------+-----+
|  -1.3598071336738|  0.0|
|  1.19185711131486|  0.0|
| -1.35835406159823|  0.0|
|-0.966271711572087|  0.0|
| -1.15823309349523|  0.0|
|-0.425965884412454|  0.0|
|  1.22965763450793|  0.0|
|-0.644269442348146|  0.0|
| -0.89428608220282|  0.0|
| -0.33826175242575|  0.0|
+------------------+-----+
only showing top 10 rows

+-------------------+-----+
|                 V1|Class|
+-------------------+-----+
|   -2.3122265423263|  1.0|
|   -3.0435406239976|  1.0|
|  -2.30334956758553|  1.0|
|  -4.39797444171999|  1.0|
|   1.23423504613468|  1.0|
|0.00843036489558254|  1.0|
| 0.0267792264491516|  1.0|
|  0.329594333318222|  1.0|
|  0.316459000444982|  1.0|
|  0.725645739819857|  1.0|
+-------------------+-----+
only showing top 10 rows



In [8]:
val time0 = trainingData0.select("Time")
val time1 = trainingData1.select("Time")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time0: org.apache.spark.sql.DataFrame = [Time: double]
time1: org.apache.spark.sql.DataFrame = [Time: double]


In [9]:
time0.describe("Time").show()
time1.describe("Time").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|             Time|
+-------+-----------------+
|  count|           198889|
|   mean|70433.87985258109|
| stddev|34483.90351654142|
|    min|              0.0|
|    max|         132881.0|
+-------+-----------------+

+-------+-----------------+
|summary|             Time|
+-------+-----------------+
|  count|              196|
|   mean|33104.29081632653|
| stddev|15821.50877735481|
|    min|            406.0|
|    max|          58222.0|
+-------+-----------------+



In [10]:
val time0v = validationData0.select("Time")
val time1v = validationData1.select("Time")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time0v: org.apache.spark.sql.DataFrame = [Time: double]
time1v: org.apache.spark.sql.DataFrame = [Time: double]


In [11]:
time0v.describe("Time").show()
time1v.describe("Time").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|              Time|
+-------+------------------+
|  count|             85426|
|   mean| 151656.3990471285|
| stddev|10987.155988334218|
|    min|          132882.0|
|    max|          172792.0|
+-------+------------------+

+-------+------------------+
|summary|              Time|
+-------+------------------+
|  count|               296|
|   mean|112293.87837837837|
| stddev| 33710.51121210961|
|    min|           58642.0|
|    max|          170348.0|
+-------+------------------+



In [12]:
val trainingData = trainingData0.unionAll(trainingData1)
val validationData = validationData0.unionAll(validationData1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [13]:
println(" Training set statistics: 1 represents fraud and 0 represents normal")
trainingData.groupBy("Class").count().show()
println(" validation set statistics: 1 represents fraud and 0 represents normal")
validationData.groupBy("Class").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Training set statistics: 1 represents fraud and 0 represents normal
+-----+------+
|Class| count|
+-----+------+
|  0.0|198889|
|  1.0|   196|
+-----+------+

 validation set statistics: 1 represents fraud and 0 represents normal
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+



In [14]:
//Gradient-boosted tree classifier
// Train a GBT model.
//setImpurity("entropy") 
val t = System.nanoTime
val gbt = new GBTClassifier().setLabelCol("label").setFeaturesCol("features").setMaxIter(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 636739730940560
gbt: org.apache.spark.ml.classification.GBTClassifier = gbtc_846e609dcd91


In [15]:
val modelGBT = gbt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelGBT: org.apache.spark.ml.classification.GBTClassificationModel = GBTClassificationModel (uid=gbtc_846e609dcd91) with 10 trees
durationtrain: Double = 37.54036023

initial model training finished.
Training process takes 37.54036023 secs


In [16]:
val s = System.nanoTime
val predictionsGBT = modelGBT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsGBT.cache()
//val predictionsAndLabel: RDD[Row] = df.rdd= predictionsGBT.select("prediction", "label")
predictionsGBT.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 636777936293515
predictionsGBT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.234111768

initial model training finished.
Training process takes 0.234111768 secs
res24: predictionsGBT.type = [V1: double, V2: double ... 35 more fields]
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [17]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsGBT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85553|
|       1.0|  169|
+----------+-----+



In [18]:
println(s"Matrice de confusion :")
predictionsGBT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85419|
|       1.0|  0.0|    7|
|       0.0|  1.0|  134|
|       1.0|  1.0|  162|
+----------+-----+-----+



In [19]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsGBT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsGBT))
println("Accuracy = " + evaluator1.evaluate(predictionsGBT))
println("Precision = " + evaluator2.evaluate(predictionsGBT))
println("Recall = " + evaluator3.evaluate(predictionsGBT))
println("F1 = " + evaluator4.evaluate(predictionsGBT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_0a03d949d6b9
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6090f896a291
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9485bf7d2822
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5d1bf5b86b85
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_8b5bc8bdd5a1
accuracy: Double = 0.9983551480366767
Area Under ROC Curve = 0.7736076775157383
Accuracy = 0.9983551480366767
Precision = 0.9982961032477127
Recall = 0.9983551480366767
F1 = 0.9981311393501129
Test Error = 0.0016448519633233127


In [20]:
// Random forest classifier
// Train a RandomForest model.
val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(10)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_43a2c0386e85
t: Long = 636793081459956


In [21]:
val modelRF = rf.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelRF: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_43a2c0386e85) with 10 trees
durationtrain: Double = 6.745513351

initial model training finished.
Training process takes 6.745513351 secs


In [22]:
val s = System.nanoTime
val predictionsRF = modelRF.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsRF.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 636800985705576
predictionsRF: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.223474162

initial model training finished.
Training process takes 0.223474162 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [23]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsRF.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85528|
|       1.0|  194|
+----------+-----+



In [24]:
println(s"Matrice de confusion :")
predictionsRF.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85425|
|       1.0|  0.0|    1|
|       0.0|  1.0|  103|
|       1.0|  1.0|  193|
+----------+-----+-----+



In [25]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsRF)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsRF))
println("Accuracy = " + evaluator1.evaluate(predictionsRF))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ee854dfb8554
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_292cbcb70243
accuracy: Double = 0.9987867758568395
Area Under ROC Curve = 0.8260076604945262
Accuracy = 0.9987867758568395
Test Error = 0.0012132241431604962


In [26]:
// Decision Tree classifier
// Train a DecisionTree model.
val dt = new DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features")
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_a85417e03393
t: Long = 636812134080701


In [27]:
val modelDT = dt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelDT: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_a85417e03393) of depth 5 with 23 nodes
durationtrain: Double = 4.540188556

initial model training finished.
Training process takes 4.540188556 secs


In [28]:
val s = System.nanoTime
val predictionsDT = modelDT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsDT.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 636817975659271
predictionsDT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.210284419

initial model training finished.
Training process takes 0.210284419 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [29]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsDT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85518|
|       1.0|  204|
+----------+-----+



In [30]:
println(s"Matrice de confusion :")
predictionsDT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85420|
|       1.0|  0.0|    6|
|       0.0|  1.0|   98|
|       1.0|  1.0|  198|
+----------+-----+-----+



In [31]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsDT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsDT))
println("Accuracy = " + evaluator1.evaluate(predictionsDT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_b991973593b2
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_abb294db7727
accuracy: Double = 0.9987867758568395
Area Under ROC Curve = 0.8344243413455362
Accuracy = 0.9987867758568395
Test Error = 0.0012132241431604962


In [32]:
// SVM classifier
// Train a SVM model.
val svm = new LinearSVC().setLabelCol("label").setFeaturesCol("features").setMaxIter(100).setRegParam(0.1) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

svm: org.apache.spark.ml.classification.LinearSVC = linearsvc_b74435837257
t: Long = 636826097677429


In [33]:
val modelSVM = svm.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelSVM: org.apache.spark.ml.classification.LinearSVCModel = linearsvc_b74435837257
durationtrain: Double = 126.828763778

initial model training finished.
Training process takes 126.828763778 secs


In [34]:
val s = System.nanoTime
val predictionsSVM = modelSVM.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsSVM.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 636955504205739
predictionsSVM: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationprediction: Double = 0.207127827

initial model training finished.
Training process takes 0.207127827 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [35]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsSVM.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85693|
|       1.0|   29|
+----------+-----+



In [36]:
println(s"Matrice de confusion :")
predictionsSVM.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85426|
|       0.0|  1.0|  267|
|       1.0|  1.0|   29|
+----------+-----+-----+



In [37]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsSVM)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsSVM))
println("Accuracy = " + evaluator1.evaluate(predictionsSVM))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6a97242af936
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_d857cf3aff35
accuracy: Double = 0.9968852803247708
Area Under ROC Curve = 0.5489864864864865
Accuracy = 0.9968852803247708
Test Error = 0.0031147196752292494


In [38]:
// Logistic Regression classifier
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_22f97c305811
t: Long = 636963689152131


In [39]:
val Modellr = lr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Modellr: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_22f97c305811, numClasses = 2, numFeatures = 11
durationtrain: Double = 5.387948924

initial model training finished.
Training process takes 5.387948924 secs


In [40]:
val s = System.nanoTime
val predictionsLR = Modellr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsLR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 636969579548502
predictionsLR: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.206119929

initial model training finished.
Training process takes 0.206119929 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [41]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsLR.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85640|
|       1.0|   82|
+----------+-----+



In [42]:
println(s"Matrice de confusion :")
predictionsLR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85426|
|       0.0|  1.0|  214|
|       1.0|  1.0|   82|
+----------+-----+-----+



In [43]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsLR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsLR))
println("Accuracy = " + evaluator1.evaluate(predictionsLR))
println("Precision = " + evaluator2.evaluate(predictionsLR))
println("Recall = " + evaluator3.evaluate(predictionsLR))
println("F1 = " + evaluator4.evaluate(predictionsLR))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_c963ea157d65
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_8f1c2bd8ecb0
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6e2f6827358c
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_c0a03dc4a986
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_dd7c10fd2fce
accuracy: Double = 0.9975035580131122
Area Under ROC Curve = 0.6385135135135135
Accuracy = 0.9975035580131122
Precision = 0.9975097962030375
Recall = 0.9975035580131121
F1 = 0.9967984547163248
Test Error = 0.0024964419868878096


In [44]:
// Naive Bayes classifier
// Train a NaiveBayes model.
val nb = new NaiveBayes().setLabelCol("label").setFeaturesCol("features").setSmoothing(1.0)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nb: org.apache.spark.ml.classification.NaiveBayes = nb_bdf70aa3096b
t: Long = 636981755802615


In [45]:
/*val scaler = new MinMaxScaler().setInputCol("features").setOutputCol("featuresScaled")
val pipeline = new Pipeline().setStages(Array(scaler))
val pipelineModel1 = pipeline.fit(trainingData)
val pipelineModel2 = pipeline.fit(trainingData)
val datatrain = pipelineModel1.transform(trainingData)
val datatest = pipelineModel2.transform(validationData)*/

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
val modelNB = nb.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelNB: org.apache.spark.ml.classification.NaiveBayesModel = NaiveBayesModel (uid=nb_bdf70aa3096b) with 2 classes
durationtrain: Double = 2.100122531

initial model training finished.
Training process takes 2.100122531 secs


In [47]:
val s = System.nanoTime
val predictionsNB = modelNB.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsNB.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 636984371828945
predictionsNB: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.207262388

initial model training finished.
Training process takes 0.207262388 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [48]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsNB.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85722|
+----------+-----+



In [49]:
println(s"Matrice de confusion :")
predictionsNB.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85426|
|       0.0|  1.0|  296|
+----------+-----+-----+



In [50]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsNB)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsNB))
println("Accuracy = " + evaluator1.evaluate(predictionsNB))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_3bc1acbe507f
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4ebabfcb91c5
accuracy: Double = 0.9965469774386971
Area Under ROC Curve = 0.5
Accuracy = 0.9965469774386971
Test Error = 0.0034530225613028653


In [51]:
// Multilayer Perceptron Classifier
// create the trainer and set its parameters
val t = System.nanoTime
val layers = Array[Int] (11, 6, 3, 2)
val mlp = new MultilayerPerceptronClassifier().setLayers(layers).setLabelCol("label").setFeaturesCol("features").setTol(1E-4).setBlockSize(128).setSeed(1234L).setMaxIter(25) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 636991377607345
layers: Array[Int] = Array(11, 6, 3, 2)
mlp: org.apache.spark.ml.classification.MultilayerPerceptronClassifier = mlpc_1d16413b8351


In [52]:
// Train a Multilayer Perceptron model.
val modelMLP = mlp.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelMLP: org.apache.spark.ml.classification.MultilayerPerceptronClassificationModel = mlpc_1d16413b8351
durationtrain: Double = 17.777745942

initial model training finished.
Training process takes 17.777745942 secs


In [53]:
val s = System.nanoTime
val predictionsMLP = modelMLP.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsMLP.select("prediction","label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 637011577853729
predictionsMLP: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.210092651

initial model training finished.
Training process takes 0.210092651 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [54]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsMLP.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85461|
|       1.0|  261|
+----------+-----+



In [55]:
println(s"Matrice de confusion :")
predictionsMLP.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85383|
|       1.0|  0.0|   43|
|       0.0|  1.0|   78|
|       1.0|  1.0|  218|
+----------+-----+-----+



In [56]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsMLP)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsMLP))
println("Accuracy = " + evaluator1.evaluate(predictionsMLP))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6de3106ce602
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_a4e645ccf199
accuracy: Double = 0.9985884603718999
Area Under ROC Curve = 0.867991563426794
Accuracy = 0.9985884603718999
Test Error = 0.0014115396281001447


In [57]:
// One Vs Rest Classifier using Logistic Regression classifier
val classifier = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

classifier: org.apache.spark.ml.classification.LogisticRegression = logreg_0477103bc783
t: Long = 637021696109544


In [58]:
// train the multiclass model.
val ovr = new OneVsRest().setClassifier(classifier)
val Modelovr = ovr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ovr: org.apache.spark.ml.classification.OneVsRest = oneVsRest_0961c1432dd4
Modelovr: org.apache.spark.ml.classification.OneVsRestModel = oneVsRest_0961c1432dd4
durationtrain: Double = 10.461463569

initial model training finished.
Training process takes 10.461463569 secs


In [59]:
val s = System.nanoTime
val predictionsOVR = Modelovr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsOVR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 637033717231520
predictionsOVR: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationprediction: Double = 0.373271085

initial model training finished.
Training process takes 0.373271085 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [60]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsOVR.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  296|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85640|
|       1.0|   82|
+----------+-----+



In [61]:
println(s"Matrice de confusion :")
predictionsOVR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85426|
|       0.0|  1.0|  214|
|       1.0|  1.0|   82|
+----------+-----+-----+



In [62]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val areaUnderPR = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderPR")
val accuracy = evaluator1.evaluate(predictionsOVR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsOVR))
println("Area Under the Precision-Recall Curve = "  + areaUnderPR.evaluate(predictionsOVR))
println("Accuracy = " + evaluator1.evaluate(predictionsOVR))
println("Precision = " + evaluator2.evaluate(predictionsOVR))
println("Recall = " + evaluator3.evaluate(predictionsOVR))
println("F1 = " + evaluator4.evaluate(predictionsOVR))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_92967a157df9
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_04ef478657a6
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_a54ed3f506ed
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_b21475277757
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_78a2d20b61c4
areaUnderPR: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_f96d4ad886f7
accuracy: Double = 0.9975035580131122
Area Under ROC Curve = 0.6385135135135135
Area Under the Precision-Recall Curve = 0.6397617345069575
Accuracy = 0.9975035580131122
Precision = 0.9975097962030375
Recall = 0.9975035580131121
F1 = 0.9967984547163248
Test Error = 0.0024964419868878096
